In [1]:
import torch
from torchvision.transforms import v2
from torch.utils.data import DataLoader

from project.data.luna_dataset import Luna16Dataset
from project.vnet import VNet

D:\Code\CISC3027\project\src\project\vnet\vnet.py:45: SyntaxWarning: invalid escape sequence '\s'
  """


In [2]:
model = VNet(num_classes=1)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model.to(device)

VNet(
  (input_block): VNet_input_block(
    (conv1): Conv3d(1, 16, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
    (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): PReLU(num_parameters=16)
  )
  (down_block1): VNet_down_block(
    (down_conv): Conv3d(16, 32, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): PReLU(num_parameters=32)
    (convs): Sequential(
      (0): Conv_in_stage(
        (conv1): Conv3d(32, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
        (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): PReLU(num_parameters=1)
      )
      (1): Conv_in_stage(
        (conv1): Conv3d(32, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
        (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [5]:
from project.config import PROJECT_ROOT

transforms = v2.Compose([
    v2.ToDtype(torch.float32, scale=True),
])

luna16 = Luna16Dataset(root=PROJECT_ROOT / "data/luna16", transforms=transforms, train=True)
luna16_base = Luna16Dataset(root=PROJECT_ROOT / "data/luna16", transforms=None, train=True)

In [6]:
transforms(luna16_base[0])

(Image([[[[-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          ...,
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923]],
 
         [[-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          ...,
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923]],
 
         [[-0.0923, -0.0923, -0.0923,  ..., -0.0923, -0.0923, -0.0923],
          [-0.0923, -0.0923,

In [7]:
len(luna16.masks)

712

In [8]:
luna16[0][0].shape, luna16[0][1].shape

(torch.Size([1, 121, 512, 512]), torch.Size([1, 121, 512, 512]))

In [9]:
luna_train_loader = DataLoader(luna16, batch_size=1, shuffle=True)  # Batch size has to be 1 because each image has different depth

In [10]:
for i, (data, target) in enumerate(luna_train_loader):
    print(data.shape, data.dtype)
    print(target.shape, target.dtype)
    break

torch.Size([1, 1, 171, 512, 512]) torch.float32
torch.Size([1, 1, 171, 512, 512]) torch.uint8


In [14]:
from project.unet3d import UNet3D

model = UNet3D(n_channels=1, n_classes=1).to(device)

In [15]:
# Train loop

import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    for i, (data, target) in enumerate(luna_train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Batch: {i}, Loss: {loss.item()}")


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.16 GiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 4.93 GiB is allocated by PyTorch, and 61.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)